In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
import warnings 
warnings.filterwarnings('ignore')

In [12]:
df = pd.read_csv(r"C:\Users\Asus\Downloads\titanic train (2).csv",usecols=['Age','Fare','Survived'])

In [13]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [14]:
df['Age'].fillna(df['Age'].mean(),inplace=True)

In [15]:

X = df.iloc[:,1:]
y = df.iloc[:,0]

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
clf = DecisionTreeClassifier()

In [18]:
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [19]:
accuracy_score(y_test,y_pred)

0.6759776536312849

In [20]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

np.float64(0.6633083645443196)

In [21]:
kbin_age = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')

In [22]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [23]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [24]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42      ,  8.4       , 18.        , 21.        , 23.        ,
              25.        , 28.        , 29.69911765, 31.        , 34.        ,
              38.        , 44.        , 51.        , 80.        ])            ],
      dtype=object)

In [25]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42      ,  8.4       , 18.        , 21.        , 23.        ,
              25.        , 28.        , 29.69911765, 31.        , 34.        ,
              38.        , 44.        , 51.        , 80.        ])            ],
      dtype=object)

In [26]:
output = pd.DataFrame({
    'age':X_train['Age'],
    'age_trf':X_train_trf[:,0],
    'fare':X_train['Fare'],
    'fare_trf':X_train_trf[:,1]
})


In [27]:
output['age_labels'] = pd.cut(x=X_train['Age'],
                                    bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=X_train['Fare'],
                                    bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

In [28]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
99,34.0,9.0,26.0000,10.0,"(31.0, 34.0]","(21.045, 26.0]"
194,44.0,11.0,27.7208,10.0,"(38.0, 44.0]","(26.0, 29.355]"
460,48.0,11.0,26.5500,10.0,"(44.0, 51.0]","(26.0, 29.355]"
113,20.0,2.0,9.8250,5.0,"(18.0, 21.0]","(8.662, 10.5]"
386,1.0,0.0,46.9000,12.0,"(0.42, 8.4]","(39.688, 65.0]"


In [29]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

In [30]:
accuracy_score(y_test,y_pred2)

0.6312849162011173

In [32]:
df = pd.read_csv(r"C:\Users\Asus\Downloads\titanic train (2).csv")[['Age','Fare','SibSp','Parch','Survived']]

In [33]:
df.dropna(inplace=True)

In [34]:
df['family'] = df['SibSp'] + df['Parch']

In [35]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [36]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [37]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [38]:
# Without binarization

clf = DecisionTreeClassifier()

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_test,y_pred)

0.6293706293706294

In [39]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

np.float64(0.6485133020344288)

In [40]:
# Applying Binarization

from sklearn.preprocessing import Binarizer

In [41]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])
],remainder='passthrough')

In [42]:

X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [43]:
pd.DataFrame(X_train_trf,columns=['family','Age','Fare'])

,family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [44]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

accuracy_score(y_test,y_pred2)

0.6013986013986014

In [45]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

np.float64(0.6290101721439749)